In [1]:
import konlpy
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [2]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [3]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    josa = ''
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [4]:
## PaPago로 보그체 만들기
def Papago_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exclamation']


    client_id = "DItIw9wyPPPRDPx4Hy46"
    client_secret = "L6ZEleX9Mp"
    url = "https://openapi.naver.com/v1/papago/n2mt"
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()

    vogulized_sentence = ''


    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if (Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt) and (len(Toknized_Korean_Okt[word][w_divided][0]) >= 1):
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])
                    Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Okt[word][w_divided][0])
                    data = "source=ko&target=en&text=" + Toknized_Text_English
                    request = urllib.request.Request(url)
                    request.add_header("X-Naver-Client-Id", client_id)
                    request.add_header("X-Naver-Client-Secret", client_secret)
                    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                    rescode = response.getcode()

                    if rescode==200:
                        response_body = response.read()
                        json_word = json.loads(response_body.decode("utf-8"))
                        result = json_word['message']['result']['translatedText']

                        split_english = result.split(sep=' ')

                        konglish = ''

                        for w in range(len(split_english)):
                            payload = {'input':split_english[w]}
                            res = sess.post(url2, data=payload)
                            konglish += res.json()['output'].replace('?','')

                        Toknized_Korean_Okt[word][w_divided][0] = konglish
                        # print(konglish)
                        result = re.sub(r'\.', '', result)
                        konglish = str(konglish + '(' + result + ')')
                        vogulized_sentence += konglish

                    else:
                        print("Error")

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '

    print()
    print(vogulized_sentence)
    print()
    print(Korean_Text_b)

In [5]:
## 구글 번역기 보그체로 바꾸기
def Google_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exclamation']


    translator = Translator(service_urls=['translate.google.co.kr'])
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()
    vogulized_sentence = ''

    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt and (len(Toknized_Korean_Okt[word][w_divided][0]) >= 1):
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    konglish = translator.translate(Toknized_Korean_Okt[word][w_divided][0], src='ko', dest='en').text
                    result2 = konglish

                    split_english = konglish.split(sep=' ')

                    konglish = ''

                    for w in range(len(split_english)):
                        payload = {'input':split_english[w]}
                        res = sess.post(url2, data=payload)
                        konglish += res.json()['output'].replace('?','')

                    Toknized_Korean_Okt[word][w_divided][0] = konglish
                    konglish = str(konglish + '(' + result2 + ')')
                    # print(konglish)

                    vogulized_sentence += konglish

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '

    print()
    print(vogulized_sentence)
    print()
    print(Korean_Text_b)

In [8]:
if __name__ == "__main__":

    print("보그체로 변환하고 싶은 문장을 입력하세요.")
    Korean_Text_b = input()
    Korean_Text = Korean_Text_b.split(sep=' ')

    Toknize_Okt = konlpy.tag.Okt()
    Toknized_Korean_Okt = []


    for i in range(len(Korean_Text)):
        Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))

        for j in range(len(Toknized_Korean_Okt[i])):
            Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    print("보그체로 변환할 때 사용할 번역기의 숫자를 입력하세요. (1: 네이버 파파고, 2: 구글 번역기)")
    num = int(input())
    while num != 1 or num != 2:
        if num == 1:
            print()
            print("네이버 파파고를 선택했습니다.")
            Papago_Vogulize(Toknized_Korean_Okt)
            break

        elif num == 2:
            print()
            print("구글 번역기를 선택했습니다.")
            Google_Vogulize(Toknized_Korean_Okt)
            break


        else:
            print("잘못 입력했습니다. 다시 선택해 주세요. (1: 네이버 파파고, 2: 구글 번역기)")
            num = int(input())

보그체로 변환하고 싶은 문장을 입력하세요.
보그체로 변환할 때 사용할 번역기의 숫자를 입력하세요. (1: 네이버 파파고, 2: 구글 번역기)

구글 번역기를 선택했습니다.

유(you)의 엑시스턴스(existence)는 내게 스트렌스(strength)를 줍니다. 

당신의 존재는 내게 힘을 줍니다.


In [33]:
# 나의 친구가 되어주셔서 고맙습니다. 내가 지쳐 쓰러져 있을 때, 당신의 존재는 내게 힘을 줍니다. 내가 어쩔 줄 몰라 뭔가 찾고 있을 때 당신의 존재는 내게 해답을 줍니다. 내가 행복하고 신이 날 때, 당신의 존재는 내게 행복의 그램 수를 높여줍니다. 부디 영원히 머물러 주십시요. 나의 삶 속에.